In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from mnist_example import MNISTModel, MNISTDataModule

datamodule = MNISTDataModule(batch_size=10)
datamodule.setup()

In [3]:
from energizer.datamodule import ActiveDataModule

dm = ActiveDataModule(datamodule=datamodule)
dm.setup()
dm.setup_folds()

In [4]:
datamodule.test_dataloader()

In [5]:
# print(
#     len(dm.pool_fold), 
#     len(dm.test_dataloader()),
#     len(dm.train_fold), 
#     len(dm.train_dataloader()),
# )
# dm.label(list(range(100)))
# print(
#     len(dm.pool_fold), 
#     len(dm.test_dataloader()),
#     len(dm.train_fold), 
#     len(dm.train_dataloader()),
# )

---

In [6]:
import torch
from torch.utils.data import DataLoader
from energizer.strategies.strategies import EntropyStrategy
from energizer.strategies.inference import MCDropout, Deterministic
from pytorch_lightning import LightningModule, Trainer as pl_Trainer
from energizer.trainer import Trainer

In [7]:
test_dl = DataLoader(
    [datamodule.mnist_test[i] for i in range(10)], 
    shuffle=False, 
    batch_size=2,
)
x, y = next(iter(test_dl))

In [8]:
model = MNISTModel()
adapter = MCDropout(model)
strategy = EntropyStrategy(adapter, 3)
dm = ActiveDataModule(datamodule=datamodule)
dm.setup()
dm.setup_folds()

In [9]:
len(dm.pool_fold), len(dm.train_fold)

(100, 0)

In [10]:
datamodule.train_dataloader().batch_size

10

In [11]:
trainer = Trainer(max_epochs=1, enable_progress_bar=True, total_budget=3)
trainer.active_fit(strategy, datamodule=dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name                | Type           | Params
-------------------------------------------------------
0 | adapter             | MCDropout      | 184 K 
1 | accumulation_metric | AccumulateTopK | 0     
-------------------------------------------------------
184 K     Trainable params
0         Non-trainable params
184 K     Total params
0.738     Total estimated model params size (MB)


/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Pool Evaluation DataLoader 0:   0%|          | 0/10 [00:00<?, ?it/s]
batch_idx: 0
len: 100
requires grad: True True
current batch_size: 10
current_indices: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
size: tensor(10)
all_scores: tensor([  -inf,   -inf,   -inf, 2.2734, 2.2636, 2.2928, 2.2638, 2.2684, 2.2820,
        2.2802, 2.2799, 2.2611, 2.2778], grad_fn=<CatBackward0>)
all_indices: tensor([-1, -1, -1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9])
top_scores: tensor([2.2928, 2.2820, 2.2802], grad_fn=<TopkBackward0>)
top_indices: tensor([2, 5, 6]) 

Pool Evaluation DataLoader 0:  10%|█         | 1/10 [00:00<00:00, 17.27it/s]
batch_idx: 1
len: 100
requires grad: True True
current batch_size: 10
current_indices: tensor([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
size: tensor(20)
all_scores: tensor([2.2928, 2.2820, 2.2802, 2.2630, 2.2674, 2.2586, 2.2328, 2.2534, 2.2448,
        2.2757, 2.2802, 2.2662, 2.2753], grad_fn=<CatBackward0>)
all_indices: tensor([ 2,  5,  6, 10, 11, 12, 13, 14, 15, 16, 17, 18

/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1931: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,



compute indices: tensor([39,  2, 88])
Epoch 0:   9%|▉         | 1/11 [00:00<00:00, 101.70it/s, loss=nan, v_num=296]

/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/core/lightning.py:631: UserWarning: `training_step` must be implemented to be used with the Lightning Trainer
  rank_zero_warn("`training_step` must be implemented to be used with the Lightning Trainer")
/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/loops/optimization/optimizer_loop.py:137: UserWarning: `training_step` returned `None`. If this was on purpose, ignore this warning...
  self.warning_cache.warn("`training_step` returned `None`. If this was on purpose, ignore this warning...")


Epoch 0: 100%|██████████| 11/11 [00:00<00:00, 92.34it/s, loss=nan, v_num=296] 


In [20]:
len(dm.val_dataloader())

10

In [25]:
trainer.fit_loop.done

True

In [22]:
trainer.fit(strategy, datamodule=datamodule)


  | Name                | Type           | Params
-------------------------------------------------------
0 | adapter             | MCDropout      | 184 K 
1 | accumulation_metric | AccumulateTopK | 0     
-------------------------------------------------------
184 K     Trainable params
0         Non-trainable params
184 K     Total params
0.738     Total estimated model params size (MB)


In [15]:
trainer.current_epoch

1